In [12]:
import itertools
import os

import cv2
import numpy as np
import oct2py

from nemo.data.preprocess.image import center_crop, max_min_scale
from nemo.model.analysis.feature_visualization import write_complex_cell_strfs
from nemo.model.openpv_utils import np_to_pvp_shared_weight_file, read_complex_cell_weight_files

## Create Spatio-Temporal Gabor Filter Bank

### Filter Bank Parameters

In [2]:
speeds = [0, 0.25, 0.5, 1.0]
orientations = list(range(-330, 390, 30))
phases = [0, np.pi / 2]
envelope_speeds = [0, 0.25, 0.5, 1.0]
bandwidths = [3.0]
size = 48
cyc_per_degs = [0.02, 0.04, 0.09, 0.18]
size_cyc_per_deg = 0.035
n_frames = 9

In [3]:
def crop_kernel(k):
    ''' Crop moving gabor kernels such that the gabor moves from one end to the other. '''
    
    ksize = k.shape[0]
    
    std_time = np.std(k, -1)
    mean_std = np.mean(std_time)
    std_time[std_time < mean_std] = 0.0
    colsum = np.sum(std_time, 0)
    rowsum = np.sum(std_time, 1)
    r_above = rowsum[:k.shape[0] // 2]
    r_below = rowsum[k.shape[0] // 2:]
    c_left = colsum[:k.shape[1] // 2]
    c_right = colsum[k.shape[1] // 2:]
    zero_above = r_above[r_above == 0].size
    zero_below = r_below[r_below == 0].size
    zero_left = c_left[c_left == 0].size
    zero_right = c_right[c_right == 0].size
    
    start_r = max(zero_above, 0)
    start_c = max(zero_left, 0)
    end_r = min(ksize - zero_below + 2, ksize)
    end_c = min(ksize - zero_right + 2, ksize)
    k = k[start_r:end_r, start_c:end_c]
    
    if k.shape[0] < ksize:
        h_diff = ksize - k.shape[0]
        pad_h = (int(np.ceil(h_diff / 2)), int(np.floor(h_diff / 2)))
        k = np.pad(k, (pad_h, (0, 0), (0, 0)))
    if k.shape[1] < ksize:
        w_diff = ksize - k.shape[1]
        pad_w = (int(np.ceil(w_diff / 2)), int(np.floor(w_diff / 2)))
        k = np.pad(k, ((0, 0), pad_w, (0, 0)))
    
    return k

In [4]:
def create_spatio_temporal_gabor_pyramid(speeds, orientations, phases, envelope_speeds, 
                                         bandwidths, size, n_frames, cyc_per_degs, size_cyc_per_deg):
    ''' Creates a bank of spatio-temporal gabors. '''
    
    fparam_combos = itertools.product(speeds, orientations, phases, envelope_speeds, bandwidths)
    fbank = np.zeros([0, size, size, n_frames])
    
    for fnum, (speed, orientation, phase, envelope_speed, bandwidth) in enumerate(fparam_combos):
        kernel = oct2py.octave.GaborKernel3d(
            speed, 
            orientation,
            phase,
            envelope_speed,
            bandwidth
        )
        if np.amax(kernel) < 1e-16: continue
        temp_start = kernel.shape[-1] // 2 - 1
        kernel = kernel[:-1, :-1, temp_start:temp_start + n_frames]
        
        if speed > 0 and envelope_speed > 0:
            kernel = crop_kernel(kernel)

        for i, cyc_per_deg in enumerate(cyc_per_degs):
            new_size = int(cyc_per_deg * size / size_cyc_per_deg)
            kernel_resize = cv2.resize(kernel, (new_size, new_size))
            
            if kernel_resize.shape[0] > size:
                kernel_resize = center_crop(kernel_resize, size, size)
            elif kernel_resize.shape[0] < size:
                pad = size - kernel_resize.shape[0]
                kernel_resize = np.pad(
                    kernel_resize,
                    (
                        (int(np.ceil(pad / 2)), int(np.floor(pad / 2))),
                        (int(np.ceil(pad / 2)), int(np.floor(pad / 2))),
                        (0, 0)
                    )
                )
                
            if kernel_resize.shape[-1] < n_frames:
                pad = n_frames - kernel_resize.shape[-1]
                kernel_resize = np.pad(
                    kernel_resize,
                    (
                        (0, 0),
                        (0, 0),
                        (int(np.ceil(pad / 2)), int(np.floor(pad / 2)))
                    )
                )
                
            fbank = np.concatenate((fbank, kernel_resize[None, ...]))
            
    repeats = []
    for i in range(fbank.shape[0]):
        for j in range(i + 1, fbank.shape[0]):
            if j in repeats: continue
            if np.sum(fbank[i] - fbank[j]) == 0:
                repeats.append(j)
        
    fbank = np.delete(fbank, repeats, 0)
    fbank = max_min_scale(fbank) * .2 - .1
    
    return fbank

In [5]:
fbank = create_spatio_temporal_gabor_pyramid(
    speeds, 
    orientations, 
    phases, 
    envelope_speeds, 
    bandwidths, 
    size, 
    n_frames,
    cyc_per_degs,
    size_cyc_per_deg
)

In [6]:
fbank.shape

(1343, 48, 48, 9)

### Writing the Filters to Disk as PVP Files

In [7]:
tensors = [np.expand_dims(fbank[..., frame].transpose([2, 1, 0]), 2) for frame in range(fbank.shape[-1])]

In [8]:
fpaths = [os.path.join('Gabors', 'S1ToFrame{}ReconError_W.pvp'.format(frame)) for frame in range(fbank.shape[-1])]

In [9]:
np_to_pvp_shared_weight_file(tensors, fpaths, '/home/mteti/OpenPV/mlab/util')

### Check the Written PVP Weight Files by Viewing the Weights

In [13]:
pvp_fpaths = [os.path.join('Gabors', f) for f in os.listdir('Gabors')]
pvp_fpaths.sort()
weight_tensors = read_complex_cell_weight_files(pvp_fpaths)
write_complex_cell_strfs(weight_tensors, 'features.gif')